### Test sur fichiers L47AII

La conversion d'un fichier en encodage utf-8 prend du temps, mais toutes les requêtes SQL avec duckdb fonctionnent pour les gros fichiers contrairement à certaines requêtes effectuées sur Java Spark SQL et le temps d'exécution des requêtes sur DuckDB est largement plus rapide que sur Java Spark SQL

Date : 12-04-2024

Éditeur : Laurent Reynaud

Le traitement porte ici sur l'identité des clients : montant des espèces payés, quantité de marchandises achetées

In [1]:
import duckdb
from skimpy import skim

In [2]:
# Version de duckdb
print(duckdb.__version__)

0.10.1


### Fichiers Excel à charger...

In [ ]:
# # Installation et extension de spatial pour les fichiers Excel
# duckdb.install_extension("spatial")
# duckdb.load_extension("spatial")

In [ ]:
# # Lecture d'un fichier Excel avec en-tête en première ligne
# # https://duckdb.org/docs/guides/import/excel_import.html
# %sql SELECT * FROM st_read('data/Quantites.xlsx', open_options = ['HEADERS=FORCE']);

### Fichiers non encodés utf-8

Les fonctions ci-après sont à utiliser dans le cas où l'encodage n'est pas de type utf-8 : la librairie duckdb ne charge que les fichiers avec un encodage de type utf-8 pour les CSV -> [documentation](https://duckdb.org/docs/data/csv/overview.html)

Pour voir si un fichier a un encodage utf-8, il suffit de le charger sur sur l'application Notepad++ et voir l'encodage affiché au menu en bas de la fenêtre de cette application

Temps de conversion pour un fichier d'1 Go => pratiquement 1 heure

In [1]:
# Fonction pour détecter l'encodage d'un fichier CSV :
import chardet
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

In [6]:
# Fonction pour convertir le fichier CSV en encodage UTF-8 :
def convert_to_utf8(input_file, output_file):
    encoding = detect_encoding(input_file)
    with open(input_file, 'r', encoding=encoding, errors='ignore') as f_in, \
         open(output_file, 'w', newline='', encoding='utf-8', errors='ignore') as f_out:
             f_out.write(f_in.read())

In [7]:
# # Appelez ces fonctions pour convertir votre fichier CSV
# input_file = 'data/indexD001/DGITRAI222.csv'
# output_file = 'data/indexD001/DGITRAI222-utf8.csv'
# convert_to_utf8(input_file, output_file)

### Configuration de JupySQL

In [2]:
# Connexion native à DuckDB : pour saisir du SQL sur notebook jupyter
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [3]:
# Configurations sur jupysql pour afficher les données en DF
%config SqlMagic.autopandas = True

In [4]:
# Connection jupysql à DuckDB
%sql duckdb:///:default:

Connecting and switching to connection 'duckdb:///:default:'

### Requêtes SQL

Affichage des premières lignes

In [5]:
%%sql 
df << 
SELECT * FROM 'data/indexD001/DGITRAI222-utf8.csv' LIMIT 5;

Running query in 'duckdb:///:default:'

In [6]:
df.head()

,ENNUME,ENSEQU,ENCLIE,ENFOUA,ENTHT,ENTTC,ENMRGT,ENNFAC,ENTKCA,ENDFAA,...,ALNSEQ,ALTYPE,ALSEQU,ALLIGNB,CLNOM,CLADR1,CLADR2,CLPTT,CLVILL,CLCLIE
0,69854,2,ZZ009,A,-850.78,-1020.94,SCR,4811899,,22,...,0,A,6,LUTRINGER MARIE PASCALE 19 RUE PRINCIPALE 6876...,COMPTANT PART CERNAY,,,,,ZZ009
1,118936,1,ZZ074,A,-115.31,-138.37,ESP,4658617,,22,...,0,A,6,DESCHAMPS JORDAN 6 RUE DES GENEVRIERS 25660 SA...,COMPTANT ELEC THISE,,,,,ZZ074
2,118936,1,ZZ074,A,-115.31,-138.37,ESP,4658617,,22,...,0,A,6,DESCHAMPS JORDAN 6 RUE DES GENEVRIERS 25660 SA...,COMPTANT ELEC THISE,,,,,ZZ074
3,118936,1,ZZ074,A,-115.31,-138.37,ESP,4658617,,22,...,0,A,6,DESCHAMPS JORDAN 6 RUE DES GENEVRIERS 25660 SA...,COMPTANT ELEC THISE,,,,,ZZ074
4,118936,1,ZZ074,A,-115.31,-138.37,ESP,4658617,,22,...,0,A,6,DESCHAMPS JORDAN 6 RUE DES GENEVRIERS 25660 SA...,COMPTANT ELEC THISE,,,,,ZZ074


Nombre de lignes

In [7]:
%sql SELECT COUNT(*) FROM 'data/indexD001/DGITRAI222-utf8.csv';

Running query in 'duckdb:///:default:'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,1657858


Affichage des différents types de paiements et leurs montants par ordre décroissant

In [8]:
%%sql
dfpayment <<
SELECT "ENMRGT", SUM("ENTTC") AS "Montant" 
FROM read_csv('data/indexD001/DGITRAI222-utf8.csv', types={"ENTTC": "DECIMAL(10, 2)"}) 
GROUP BY "ENMRGT" 
ORDER BY "Montant" DESC;

Running query in 'duckdb:///:default:'

In [9]:
dfpayment.head()

,ENMRGT,Montant
0,TRA,8.281783e+08
1,CHE,1.574164e+08
2,VIR,1.204510e+08
3,SCR,1.073265e+08
4,CB,6.913738e+07


In [30]:
# dfpayment.to_excel('data/indexD001/2022_payment.xlsx')

Affichage des clients qui payent en espèces et à partir d'un montant >= à 5 000 € et le champ CLNOM (RAISON SOCIAL) ne contient pas le mot 'COMPTANT'

In [10]:
%%sql
dfCashAndNoComptant <<
SELECT CLCLIE, ALLIGNB, CLNOM, CLADR1, CLADR2, CLPTT, CLVILL, 
ENMRGT, SUM(ENTTC) AS Montant 
FROM read_csv('data/indexD001/DGITRAI222-utf8.csv', types={"ENTTC": "DECIMAL(10, 2)"}) 
GROUP BY CLCLIE, ALLIGNB, CLNOM, CLADR1, CLADR2, CLPTT, CLVILL, ENMRGT
HAVING ENMRGT LIKE 'ESP' 
AND Montant >= 5000 
AND CLNOM NOT LIKE '%COMPTANT%' 
ORDER BY Montant DESC;

Running query in 'duckdb:///:default:'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
skim(dfCashAndNoComptant)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 93     │ │ string      │ 8     │                                                          │
│ │ Number of columns │ 9      │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name     ┃ NA  ┃ NA %   ┃ mean    ┃ sd      ┃ p0     ┃ p25   ┃ p50     ┃ p75    ┃ p100    ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩  │
│ │ Montant         │   0 │      0 │   18000 │   30000 │   5100 │  7000 │   11000 │  16000 │  280000 │    ▇    │  │
│ └─────────────────┴─────┴────────┴─────────┴─────────┴────────┴───────┴─────────┴────────┴─────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ CLCLIE                    │       0 │          0 │                            1 │                       93 │  │
│ │ ALLIGNB                   │       0 │          0 │                          510 │                    47299 │  │
│ │ CLNOM                     │       0 │          0 │                           16 │                     1467 │  │
│ │ CLADR1                    │       0 │          0 │                           15 │                     1421 │  │
│ │ CLADR2                    │       0 │          0 │                           31 │                     2864 │  │
│ │ CLPTT                     │       0 │          0 │                            1 │                       93 │  │
│ │ CLVILL                    │       0 │          0 │                           21 │                     1959 │  │
│ │ ENMRGT                    │       0 │          0 │                            1 │                       93 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [12]:
dfCashAndNoComptant.head()

,CLCLIE,ALLIGNB,CLNOM,CLADR1,CLADR2,CLPTT,CLVILL,ENMRGT,Montant
0,SM002,...,SARL SMSA ET FRERES,15 A RUE DES TULIPES,,68110,ILLZACH,ESP,277194.81
1,BU002,...,ENT AB SANITAIRE,5 RUE EDOUARD BRANLY,,68308,ST LOUIS,ESP,72542.65
2,SA442,...,SARL SA-THERMIQUE,71 RUE DE MULHOUSE,,68390,SAUSHEIM,ESP,68723.59
3,CN002,...,SAS CN CHAUFFAGE,25 RUE DES PAYS BAS,,68310,WITTELSHEIM,ESP,57493.43
4,BA220,...,ENT BAUDRY MATHIEU,23 RUE DE RETZWILLER,,68210,ELBACH,ESP,51651.65


In [34]:
# dfCashAndNoComptant.to_excel('data/indexD001/2022_CashAndNoComptant.xlsx')

Affichage par client des articles achetées dans la limite de 1 000 quantités achetées par article

In [13]:
%%sql
dfCustomersQuantity <<
SELECT ENCLIE, ALLIGNB, CLNOM, CLADR1, CLADR2, CLPTT, CLVILL, DENART, 
DELIBE, DEUFAC, SUM(DEQLIV) AS Quantites 
FROM read_csv('data/indexD001/DGITRAI222-utf8.csv', types={"DEQLIV": "INT"})  
GROUP BY ENCLIE, ALLIGNB, CLNOM, CLADR1, CLADR2, CLPTT, CLVILL, DENART,
DELIBE, DEUFAC 
HAVING Quantites > 1000 
ORDER BY Quantites DESC

Running query in 'duckdb:///:default:'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
skim(dfCustomersQuantity)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 896    │ │ string      │ 10    │                                                          │
│ │ Number of columns │ 11     │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %   ┃ mean   ┃ sd     ┃ p0     ┃ p25    ┃ p50    ┃ p75    ┃ p100    ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩  │
│ │ Quantites        │    0 │      0 │   2400 │   2500 │   1000 │   1200 │   1600 │   2500 │   34000 │    ▇    │  │
│ └──────────────────┴──────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴─────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ ENCLIE                    │       0 │          0 │                            1 │                      896 │  │
│ │ ALLIGNB                   │       0 │          0 │                          500 │                   449846 │  │
│ │ CLNOM                     │       0 │          0 │                           15 │                    13711 │  │
│ │ CLADR1                    │       0 │          0 │                           16 │                    14205 │  │
│ │ CLADR2                    │       0 │          0 │                           30 │                    26798 │  │
│ │ CLPTT                     │       0 │          0 │                          1.1 │                      971 │  │
│ │ CLVILL                    │       0 │          0 │                           21 │                    18706 │  │
│ │ DENART                    │       0 │          0 │                          1.3 │                     1170 │  │
│ │ DELIBE                    │       0 │          0 │                           19 │                    17051 │  │
│ │ DEUFAC                    │       0 │          0 │                          1.9 │                     1725 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [15]:
dfCustomersQuantity.head()

,ENCLIE,ALLIGNB,CLNOM,CLADR1,CLADR2,CLPTT,CLVILL,DENART,DELIBE,DEUFAC,Quantites
0,UL019,...,SARL ULKU CONSTRUCTION,8 RUE DES PRES,,68040,INGERSHEIM,G03770,HOMEBRIC 20 BRIQUE BASE 500X200X299 ...,PIEC,33500.0
1,ST133,...,SARL ST CONSTRUCTION,4 BIS RUE DES COMBOTTES,,25700,VALENTIGNEY,G00266,AGGLOS 20 500X200X200 CREUX ...,PIEC,21055.0
2,BA183,...,ENT BAZZANA PATRICE,32 RUE SOUS BOIS,,25400,TAILLECOURT,G00266,AGGLOS 20 500X200X200 CREUX ...,PIEC,18602.0
3,ST260,EURL AS BTP-STN MR TASKAFA ERKAM RUE DES PYREN...,EURL AS BTP-STN,26 RUE DES GAULOIS,,68390,SAUSHEIM,,ARMATURE ASSEMBLEES ...,KILO,18084.0
4,YA010,...,SARL YALCIN CONSTRUCTION,2 RUE DES VERRIERS,,68200,MULHOUSE,G03770,HOMEBRIC 20 BRIQUE BASE 500X200X299 ...,PIEC,17711.0


In [38]:
# dfCustomersQuantity.to_excel('data/indexD001/2022_CustomersQuantity.xlsx')